## Imports


In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

from PIL import Image




## Constants

In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd

NR_CLASSES =10
VALIDATION_SIZE =10000
IMAGE_HT = 28
IMAGE_WD = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HT*IMAGE_WD*CHANNELS




## Get Data and split into Xtrain ytrain Xtest and ytest

In [3]:
train_file ='mnist_train.csv'
test_file ='mnist_test.csv'

In [4]:
X_train = np.loadtxt(train_file, delimiter = ',', dtype=int,skiprows=0,usecols=range(1,785))
X_test = np.loadtxt(test_file, delimiter = ',', dtype=int,skiprows=0,usecols=range(1,785)) 

In [5]:
X_train.shape


(60000, 784)

In [6]:
X_test.shape

(10000, 784)

In [7]:
y_train = np.loadtxt(train_file, delimiter = ',', dtype=int,skiprows=0,usecols=0)
y_test = np.loadtxt(test_file, delimiter = ',', dtype=int,skiprows=0,usecols=0) 

In [8]:
y_train.shape

(60000,)

## Data preprocessing scaling

In [9]:
#Convert labels to dense matrix
y_train = np.eye(NR_CLASSES)[y_train]
y_test = np.eye(NR_CLASSES)[y_test]

In [10]:
y_test.shape

(10000, 10)

In [11]:
y_train.shape

(60000, 10)

In [12]:
y_test.shape

(10000, 10)

In [13]:
X_train, X_test  = X_train/255.0, X_test/255.0 #values between 0 and 1 for easy input to neural network

## Create Validation set of 10000 images from training set

In [14]:
X_val = X_train[:VALIDATION_SIZE]
y_val = y_train[:VALIDATION_SIZE]

X_train = X_train[VALIDATION_SIZE:]
y_train = y_train[VALIDATION_SIZE:]

In [15]:
X_val.shape

(10000, 784)

In [16]:
y_val.shape

(10000, 10)

In [17]:
X_train.shape

(50000, 784)

In [18]:
y_train.shape

(50000, 10)

# Create a TensorFlow graph

In [19]:
X = tf.placeholder(tf.float32,shape=[None,TOTAL_INPUTS],name= 'X')
y = tf.placeholder(tf.float32,shape=[None,NR_CLASSES], name='labels')

# Creating Neural Network


### Hyperparameters

In [113]:
nr_epochs = 25
learning_rate = 0.0001

n_hidden1 = 512
n_hidden2 = 64


### Defining the loss function, optimizer and accuracy

In [114]:
#layer 1

initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1],stddev=0.1,seed=42)
w1 = tf.Variable( initial_value = initial_w1,name='w1')
initial_b1 = tf.constant(value =0.0, shape=[n_hidden1])
b1 = tf.Variable(initial_value = initial_b1, name ='b1')

layer1_in = tf.matmul(X,w1)+b1
layer1_out = tf.nn.relu(layer1_in) #activation function is relu

#Layer2

initial_w2 = tf.truncated_normal(shape=[n_hidden1,n_hidden2], stddev =0.1, seed=42)
w2 = tf.Variable(initial_value=initial_w2, name ='w2')

initial_b2 = tf.constant(value=0.0,shape=[n_hidden2])
b2 = tf.Variable(initial_value=initial_b2, name='b2')

layer2_in = tf.matmul(layer1_out,w2)+b2
layer2_out = tf.nn.relu(layer2_in)

#Output layer
initial_w3 = tf.truncated_normal(shape=[n_hidden2,NR_CLASSES], stddev =0.1, seed=42)
w3 = tf.Variable(initial_value=initial_w3, name ='w3')

initial_b3 = tf.constant(value=0.0,shape=[NR_CLASSES])
b3 = tf.Variable(initial_value=initial_b3, name='b3')

layer3_in = tf.matmul(layer2_out,w3)+b3
output = tf.nn.softmax(layer3_in)  #function used is softmax to output values between 0 an 1



In [115]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = output)) ##Loss function..run inn batches..so take average


In [116]:
# optimizer and minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)


In [117]:
# accuracy metric
correct_pred = tf.equal(tf.argmax(output,axis=1),tf.argmax(y,axis=1)) #compare labels in tensor and output from Neural network
#10 values in each row having probability between 0 1n 1..chose the highest one as the label output
metric = tf.reduce_mean(tf.cast(correct_pred,tf.float32)) #take mean value and typecast as float as output is in decimal

## Create Session and initialize values 

In [118]:
sess= tf.Session() #Create a Session
init = tf.global_variables_initializer() #initialize variables
sess.run(init) #Run the session to evaluate the variables

In [119]:
w1.eval(sess)

b1.eval(sess) #512 values
b2.eval(sess) #64 values
b3.eval(sess) #10 values


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Batching the data

In [120]:
batch_size = 1000 #run 50000 examples in batches of 1000
num_of_examples = y_train.shape[0] #50000 examples
nr_of_iterations = int(num_of_examples/batch_size) #50000/1000
index_in_epoch = 0


In [121]:
#create a batch function

def next_batch(batch_size,data,labels):
    
    global num_of_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_of_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]
    

## Training our Model


In [122]:
for epoch in range(nr_epochs):
    
    #=============training data set==================
    
    for i in range(nr_of_iterations):
        batch_x,batch_y = next_batch(batch_size=batch_size,data=X_train,labels=y_train)
        
        feed_dictionary = {X:batch_x, y: batch_y}
        sess.run(train_step,feed_dict=feed_dictionary)
       
        batch_accuracy = sess.run(fetches=[metric], feed_dict=feed_dictionary)
              
    print(f'Epochs {epoch}  \t| Training Accuracy is {batch_accuracy}')
    
     #=================Validation dataset =================  
    validation_accuracy = sess.run(fetches=[metric],feed_dict = {X:X_val,y:y_val})
    print(f'             \t| validation accuracy is {validation_accuracy}')
    
print('Done Training!!')    

Epochs 0  	| Training Accuracy is [0.638]
             	| validation accuracy is [0.6263]
Epochs 1  	| Training Accuracy is [0.804]
             	| validation accuracy is [0.7723]
Epochs 2  	| Training Accuracy is [0.845]
             	| validation accuracy is [0.8128]
Epochs 3  	| Training Accuracy is [0.86]
             	| validation accuracy is [0.8314]
Epochs 4  	| Training Accuracy is [0.872]
             	| validation accuracy is [0.8402]
Epochs 5  	| Training Accuracy is [0.876]
             	| validation accuracy is [0.8454]
Epochs 6  	| Training Accuracy is [0.879]
             	| validation accuracy is [0.8491]
Epochs 7  	| Training Accuracy is [0.881]
             	| validation accuracy is [0.8523]
Epochs 8  	| Training Accuracy is [0.882]
             	| validation accuracy is [0.8552]
Epochs 9  	| Training Accuracy is [0.884]
             	| validation accuracy is [0.8581]
Epochs 10  	| Training Accuracy is [0.885]
             	| validation accuracy is [0.86]
Epochs 11  	

In [123]:
img = Image.open('D:/Data/Downloads/5.png')
img

In [124]:
bw = img.convert('L') #Convert color image to grey scale image
bw

In [125]:
img_array = np.invert(bw) #Covert bw image to ndarray
img_array.shape

(28, 28)

In [126]:
test_img = img_array.ravel() #flatten array with np.ravel function

In [127]:
test_img.shape

(784,)

In [128]:
prediction = sess.run(fetches=tf.argmax(output,axis=1),feed_dict={X:[test_img]})

In [129]:

print(f'Prediction for a test image is {prediction}')

Prediction for a test image is [5]


In [130]:
pred2 = sess.run(fetches= metric,feed_dict={X:X_test,y:y_test})


In [131]:
print(f'Prediction accuracy is {pred2:0.2%}')

Prediction accuracy is 94.76%


In [132]:
sess.close()